## This notebook contains the data wrangling and analysis for our second driving question: Do I have more steps on average during the weekend in comparison to a weekday?

In [1]:
#import libraries we'll be using
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#import data sets
person1 = pd.read_csv('person1-step-data.csv')
person2 = pd.read_csv('person2-step-data.csv')

In [3]:
#drop na values
df1 = person1.dropna()
df2 = person2.dropna()

In [4]:
#generate datetime column from the Date and Hour columns
def gen_datetime(row):
    h = row['Hour']
    if h < 10:
        h = '-' + str(0) + str(h)
    else:
        h = '-' + str(h)
    return pd.to_datetime(row['Date'] + h, format='%Y-%m-%d-%H')

df1['Datetime'] = df1.apply(gen_datetime, axis=1)
df2['Datetime'] = df2.apply(gen_datetime, axis=1)

In [5]:
#delete Date and Hour columns
df1 = df1.drop(columns=['Date', 'Hour'])
df2 = df2.drop(columns=['Date', 'Hour'])

In [6]:
#group by Datetime
gdf1 = df1.groupby('Datetime')
gdf2 = df2.groupby('Datetime')

In [7]:
#For data set 1, sum Count variable hourly, so there are no two rows with the same date and time
keys1 = gdf1.groups.keys()
rows1 = []
for i in keys1:
    rows1.append([i, gdf1.get_group(i)['Count'].sum()])

In [8]:
#For data set 2, sum Count variable hourly, so there are no two rows with the same date and time
keys2 = gdf2.groups.keys()
rows2 = []
for i in keys2:
    rows2.append([i, gdf2.get_group(i)['Count'].sum()])

In [9]:
#Create new DataFrames from summed count data
df1_hourly = pd.DataFrame(rows1, columns=['Timestamp', 'Count'])
df2_hourly = pd.DataFrame(rows2, columns=['Timestamp', 'Count'])

In [10]:
#add Day of the Week column
def gen_dayofweek(row):
    return row['Timestamp'].weekday()
df1_hourly['Day of Week'] = df1_hourly.apply(gen_dayofweek, axis=1)
df2_hourly['Day of Week'] = df2_hourly.apply(gen_dayofweek, axis=1)

In [11]:
#add Is Weekday column
def gen_isweekday(row):
    if row['Day of Week'] < 5:
        return True
    else:
        return False
df1_hourly['Is Weekday'] = df1_hourly.apply(gen_isweekday, axis=1)
df2_hourly['Is Weekday'] = df2_hourly.apply(gen_isweekday, axis=1)

In [12]:
df1_hourly

,Timestamp,Count,Day of Week,Is Weekday
0,2014-12-07 08:00:00,941,6,False
1,2014-12-07 09:00:00,408,6,False
2,2014-12-07 10:00:00,157,6,False
3,2014-12-07 11:00:00,1017,6,False
4,2014-12-07 14:00:00,137,6,False
...,...,...,...,...
19884,2021-09-22 12:00:00,305,2,True
19885,2021-09-22 13:00:00,237,2,True
19886,2021-09-22 14:00:00,9,2,True
19887,2021-09-22 17:00:00,3560,2,True


In [13]:
df2_hourly

,Timestamp,Count,Day of Week,Is Weekday
0,2014-11-29 06:00:00,28,5,False
1,2014-11-29 12:00:00,207,5,False
2,2014-11-29 13:00:00,1621,5,False
3,2014-11-29 14:00:00,1045,5,False
4,2014-11-29 15:00:00,219,5,False
...,...,...,...,...
35169,2021-09-22 11:00:00,165,2,True
35170,2021-09-22 12:00:00,365,2,True
35171,2021-09-22 13:00:00,198,2,True
35172,2021-09-22 14:00:00,143,2,True


In [14]:
#add a Date column to a copy of the hourly DataFrame
def timestamp_to_date(row):
    return row['Timestamp'].date()
df1_daily = df1_hourly.copy()
df2_daily = df2_hourly.copy()
df1_daily['Date'] = df1_hourly.apply(timestamp_to_date, axis=1)
df2_daily['Date'] = df2_hourly.apply(timestamp_to_date, axis=1)
#remove timestamp column
df1_daily = df1_daily.drop(columns = ['Timestamp'])
df2_daily = df2_daily.drop(columns = ['Timestamp'])
#group by date
gdf1_daily = df1_daily.groupby('Date')
gdf2_daily = df2_daily.groupby('Date')

In [15]:
#For data set 1, sum Count variable daily, so there are no two rows with the same date
keys1 = gdf1_daily.groups.keys()
rows1 = []
for i in keys1:
    rows1.append([i, gdf1_daily.get_group(i)['Count'].sum(), gdf1_daily.get_group(i).iloc[0]['Day of Week'], gdf1_daily.get_group(i).iloc[0]['Is Weekday']])

In [16]:
#For data set 2, sum Count variable daily, so there are no two rows with the same date
keys2 = gdf2_daily.groups.keys()
rows2 = []
for i in keys2:
    rows2.append([i, gdf2_daily.get_group(i)['Count'].sum(), gdf2_daily.get_group(i).iloc[0]['Day of Week'], gdf2_daily.get_group(i).iloc[0]['Is Weekday']])

In [17]:
#create new dataframes from summed Count data
df1_daily = pd.DataFrame(rows1, columns=['Date', 'Count', 'Day of Week', 'Is Weekday'])
df2_daily = pd.DataFrame(rows2, columns=['Date', 'Count', 'Day of Week', 'Is Weekday'])

In [18]:
df1_daily

,Date,Count,Day of Week,Is Weekday
0,2014-12-07,2693,6,False
1,2014-12-08,6567,0,True
2,2014-12-09,6879,1,True
3,2014-12-10,7845,2,True
4,2014-12-11,916,3,True
...,...,...,...,...
2290,2021-09-18,11572,5,False
2291,2021-09-19,10789,6,False
2292,2021-09-20,8800,0,True
2293,2021-09-21,17071,1,True


In [19]:
df2_daily

,Date,Count,Day of Week,Is Weekday
0,2014-11-29,12464,5,False
1,2014-11-30,2759,6,False
2,2014-12-01,6672,0,True
3,2014-12-02,9658,1,True
4,2014-12-03,1709,2,True
...,...,...,...,...
2485,2021-09-18,15250,5,False
2486,2021-09-19,19358,6,False
2487,2021-09-20,7563,0,True
2488,2021-09-21,14232,1,True


In [20]:
#group both datasets by day of the week
gdf1_daily = df1_daily.groupby('Day of Week')
gdf2_daily = df2_daily.groupby('Day of Week')
#create keys
dkeys1 = gdf1_daily.groups.keys()
dkeys2 = gdf2_daily.groups.keys()
#create dict to translate weekday numbers to names
daydict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

In [21]:
#show mean steps per day
print('\nMean steps per day, person1')
for i in dkeys1:
    print(daydict[i], gdf1_daily.get_group(i)['Count'].mean())
print('\nMean steps per day, person2')
for i in dkeys2:
    print(daydict[i], gdf2_daily.get_group(i)['Count'].mean())


Mean steps per day, person1
Monday 6004.833827893175
Tuesday 6046.846607669617
Wednesday 6300.5645645645645
Thursday 7034.2764705882355
Friday 6424.283625730995
Saturday 6111.5331010452965
Sunday 6783.230283911672

Mean steps per day, person2
Monday 13080.457865168539
Tuesday 14603.3202247191
Wednesday 13848.317415730337
Thursday 14362.374647887324
Friday 14236.408450704226
Saturday 11692.351123595505
Sunday 15901.306179775282


In [22]:
#group both datasets by weekend or weekday
gdf1_daily = df1_daily.groupby('Is Weekday')
gdf2_daily = df2_daily.groupby('Is Weekday')
#create keys
dkeys1 = gdf1_daily.groups.keys()
dkeys2 = gdf2_daily.groups.keys()
#create dict to translate True/False to weekday/weekend
weekdict = {True:'Weekday', False:'Weekend'}

In [23]:
#show mean steps, weekday vs weekend
print('\nMean steps for weekend or weekday, person1')
for i in dkeys1:
    print(weekdict[i], gdf1_daily.get_group(i)['Count'].mean())
print('\nMean steps for weekend or weekday, person2')
for i in dkeys2:
    print(weekdict[i], gdf2_daily.get_group(i)['Count'].mean())


Mean steps for weekend or weekday, person1
Weekend 6464.062913907284
Weekday 6363.309875813128

Mean steps for weekend or weekday, person2
Weekend 13796.828651685393
Weekday 14025.86839145107
